In [15]:
import os

os.chdir("/work")

# %% 準備
import pandas as pd
from utils.logger import app_logger
from utils.config_loader import get_template_config
from logic.manage.utils.csv_loader import load_all_filtered_dataframes
from logic.manage.utils.load_template import load_master_and_template
from IPython.display import display
import re
from logic.manage.factory_report import process

In [16]:
from logic.manage.processors.factory_report_yuuka import process_yuuka

In [21]:
# 表示ラベルマップ（処理対象名として使う）
csv_label_map = {"yard": "ヤード一覧", "shipping": "出荷一覧", "receive": "受入一覧"}

debug_shipping = "/work/data/output/debug_shipping.parquet"
debug_yard = "/work/data/output/debug_yard.parquet"

dfs = {
    "shipping": pd.read_parquet(debug_shipping),
    "yard": pd.read_parquet(debug_yard),
}  # テスト用CSV
# dfs
df_shipping = dfs["shipping"]
df_yard = dfs["yard"]

In [27]:
def process_yuuka(df_yard, df_shipping) -> pd.DataFrame:
    logger = app_logger()

    # マスターCSVの読込
    master_path = get_template_config()["factory_report"]["master_csv_path"]["yuuka"]
    master_csv = load_master_and_template(master_path)

    # 各処理を実行
    # updated_master_csv = apply_shipping(master_csv, df_shipping)

    # display(master_csv)
    return master_csv
master_csv1 = process_yuuka(df_yard, df_shipping)

In [ ]:
def apply_shipping(master_csv, df_shipping):
    logger = app_logger()
    

master_csv2 = apply_shipping(master_csv1,df_shipping)